In [ ]:
## This file implements neural networks and logistic regression on p003ppresabsSTCC_quant.
## Due to the imbalanced dataset, we implement the over-sampling method and the combination of over- and under-sampling
## method.
## For fully-connected neural networks, the accuracy is 83.33% for combination data, and 81.94% for over-sampling data.
## To improve accuracy, we use lasso to select important features and thus have new training and test datasets.
## For the new neural networks on these new datasets, the accuracy is 88.19% for combination data and 84.03% for over-sampling data.
## For logistic regression, the accuracies are 100% for combination data and 97.92% for over-sampling data.
## Since the accuracy scores are pretty high in logistic regression, we further construct random forest models, which 
## are relatively less likely to bring overfitting compared to decision tree.
## For random forest, the accuracy is 99.24% for combination and 100% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 99.34% for combination data, and 98.51% for over-sampling.

In [1]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p003ppresabsSTCC_quant.csv')
df.shape

(255, 868)

In [2]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [3]:
df['pheno']

0      0.169500
1      0.316500
2      0.460000
3      0.169000
4      0.171333
5      0.236333
6      0.166500
7      0.173333
8      0.315833
9      0.162167
10     0.186167
11     0.382833
12     0.170667
13     0.173333
14     0.162833
15     0.164333
16     0.382667
17     0.397000
18     0.372667
19     0.166333
20     0.242333
21     0.305833
22     0.162333
23     0.165500
24     0.616833
25     0.170167
26     0.491500
27     0.167833
28     0.166500
29     0.374833
         ...   
225    0.267200
226    0.293800
227    0.383400
228    0.513500
229    0.507700
230    0.311500
231    0.236000
232    0.228833
233    0.219333
234    0.167333
235    0.265167
236    0.160500
237    0.159333
238    0.165500
239    0.327833
240    0.342000
241    0.308500
242    0.170333
243    0.165333
244    0.340833
245    0.162000
246    0.170500
247    0.164500
248    0.164000
249    0.180000
250    0.348000
251    0.232500
252    0.166167
253    0.276667
254    0.225333
Name: pheno, Length: 255

In [5]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [6]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [7]:
df['pheno']
df['pheno'].value_counts()

0    240
1     15
Name: pheno, dtype: int64

In [8]:
df.shape

(255, 868)

In [9]:
df_clean = df.drop(columns=['id'])

In [10]:
df_clean.shape

(255, 867)

In [11]:
df_clean.head()

,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATCTCACCAATTTTGTAATACATCGTTCTCGTCCTCCTTGTCTTCTTCGTCCTCCTCGTTATCTTCTTCGTTTTGTAATTCATAAATTTTGTTT,TTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAA,TTTTTTAGGTACC,TTTTTGCATTCA,TTTTTCTTATTTTA,...,group_8664,group_8665,group_8666,group_8667,group_8836,group_8913,group_9508,pheno,ST,CC
0,1,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,5,5
1,1,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,8,8
2,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,5,5
3,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,5,5
4,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,5,5


In [12]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 866) (255,)


In [13]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

Using TensorFlow backend.


[(0, 209), (1, 228)]


In [14]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 240), (1, 240)]


In [15]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [16]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [17]:
############# Fully-Connected Neural Network ################

In [18]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [19]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [20]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [21]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 305 samples, validate on 132 samples
Epoch 1/1000
305/305 [==============================] - 0s 512us/step - loss: 7.1623 - accuracy: 0.6852 - val_loss: 0.6109 - val_accuracy: 0.6894
Epoch 2/1000
305/305 [==============================] - 0s 126us/step - loss: 0.6132 - accuracy: 0.7049 - val_loss: 0.5790 - val_accuracy: 0.7727
Epoch 3/1000
305/305 [==============================] - 0s 156us/step - loss: 0.5963 - accuracy: 0.7213 - val_loss: 0.5739 - val_accuracy: 0.6894
Epoch 4/1000
305/305 [==============================] - 0s 120us/step - loss: 0.6189 - accuracy: 0.7246 - val_loss: 0.5801 - val_accuracy: 0.6894
Epoch 5/1000
305/305 [==============================] - 0s 220us/step - loss: 0.5791 - accuracy: 0.7049 - val_loss: 0.5444 - val_accuracy: 0.8106
Epoch 6/1000
305/305 [==============================] - 0s 119us/step - loss: 0.5679 - accuracy: 0.7574 - val_loss: 0.5714 - val_accuracy: 0.6894
Epoch 7/1000
305/305 [==============================] - 0s 121us/step - loss: 

305/305 [==============================] - 0s 157us/step - loss: 0.4656 - accuracy: 0.7574 - val_loss: 0.4237 - val_accuracy: 0.8182
Epoch 57/1000
305/305 [==============================] - 0s 176us/step - loss: 0.4654 - accuracy: 0.7574 - val_loss: 0.4112 - val_accuracy: 0.8106
Epoch 58/1000
305/305 [==============================] - 0s 126us/step - loss: 0.4631 - accuracy: 0.7607 - val_loss: 0.4121 - val_accuracy: 0.8106
Epoch 59/1000
305/305 [==============================] - 0s 148us/step - loss: 0.4598 - accuracy: 0.7574 - val_loss: 0.4057 - val_accuracy: 0.8106
Epoch 60/1000
305/305 [==============================] - 0s 107us/step - loss: 0.4603 - accuracy: 0.7574 - val_loss: 0.4083 - val_accuracy: 0.8106
Epoch 61/1000
305/305 [==============================] - 0s 124us/step - loss: 0.4588 - accuracy: 0.7574 - val_loss: 0.4045 - val_accuracy: 0.8106
Epoch 62/1000
305/305 [==============================] - 0s 107us/step - loss: 0.4559 - accuracy: 0.7574 - val_loss: 0.4039 - val_ac

305/305 [==============================] - 0s 116us/step - loss: 0.4300 - accuracy: 0.7705 - val_loss: 0.4093 - val_accuracy: 0.8182
Epoch 112/1000
305/305 [==============================] - 0s 112us/step - loss: 0.4579 - accuracy: 0.7672 - val_loss: 0.4066 - val_accuracy: 0.8258
Epoch 113/1000
305/305 [==============================] - 0s 102us/step - loss: 0.4538 - accuracy: 0.7705 - val_loss: 0.4028 - val_accuracy: 0.8182
Epoch 114/1000
305/305 [==============================] - 0s 96us/step - loss: 0.4458 - accuracy: 0.7705 - val_loss: 0.3952 - val_accuracy: 0.8182
Epoch 115/1000
305/305 [==============================] - 0s 92us/step - loss: 0.4340 - accuracy: 0.7705 - val_loss: 0.3651 - val_accuracy: 0.8182
Epoch 116/1000
305/305 [==============================] - 0s 106us/step - loss: 0.4102 - accuracy: 0.7705 - val_loss: 0.3593 - val_accuracy: 0.8106
Epoch 117/1000
305/305 [==============================] - 0s 131us/step - loss: 0.4054 - accuracy: 0.7705 - val_loss: 0.3594 - va

305/305 [==============================] - 0s 109us/step - loss: 0.3992 - accuracy: 0.7738 - val_loss: 0.3496 - val_accuracy: 0.8258
Epoch 167/1000
305/305 [==============================] - 0s 103us/step - loss: 0.3985 - accuracy: 0.7738 - val_loss: 0.3456 - val_accuracy: 0.8333
Epoch 168/1000
305/305 [==============================] - 0s 90us/step - loss: 0.3979 - accuracy: 0.7738 - val_loss: 0.3450 - val_accuracy: 0.8258
Epoch 169/1000
305/305 [==============================] - 0s 103us/step - loss: 0.3942 - accuracy: 0.7738 - val_loss: 0.3461 - val_accuracy: 0.8333
Epoch 170/1000
305/305 [==============================] - 0s 110us/step - loss: 0.3923 - accuracy: 0.7738 - val_loss: 0.3432 - val_accuracy: 0.8333
Epoch 171/1000
305/305 [==============================] - 0s 98us/step - loss: 0.3901 - accuracy: 0.7738 - val_loss: 0.3462 - val_accuracy: 0.8258
Epoch 172/1000
305/305 [==============================] - 0s 96us/step - loss: 0.3913 - accuracy: 0.7738 - val_loss: 0.3406 - val

305/305 [==============================] - 0s 168us/step - loss: 0.3794 - accuracy: 0.7738 - val_loss: 0.3368 - val_accuracy: 0.8258
Epoch 222/1000
305/305 [==============================] - 0s 96us/step - loss: 0.3790 - accuracy: 0.7738 - val_loss: 0.3354 - val_accuracy: 0.8258
Epoch 223/1000
305/305 [==============================] - 0s 103us/step - loss: 0.3785 - accuracy: 0.7738 - val_loss: 0.3366 - val_accuracy: 0.8333
Epoch 224/1000
305/305 [==============================] - 0s 98us/step - loss: 0.3771 - accuracy: 0.7738 - val_loss: 0.3323 - val_accuracy: 0.8333
Epoch 225/1000
305/305 [==============================] - 0s 105us/step - loss: 0.3723 - accuracy: 0.7738 - val_loss: 0.3410 - val_accuracy: 0.8409
Epoch 226/1000
305/305 [==============================] - 0s 104us/step - loss: 0.3763 - accuracy: 0.7738 - val_loss: 0.3220 - val_accuracy: 0.8333
Epoch 227/1000
305/305 [==============================] - 0s 157us/step - loss: 0.3678 - accuracy: 0.7738 - val_loss: 0.3133 - va

...

305/305 [==============================] - 0s 345us/step - loss: 0.4011 - accuracy: 0.7738 - val_loss: 0.3280 - val_accuracy: 0.8409
Epoch 731/1000
305/305 [==============================] - 0s 104us/step - loss: 0.3992 - accuracy: 0.7738 - val_loss: 0.3279 - val_accuracy: 0.8409
Epoch 732/1000
305/305 [==============================] - 0s 175us/step - loss: 0.3998 - accuracy: 0.7738 - val_loss: 0.3288 - val_accuracy: 0.8409
Epoch 733/1000
305/305 [==============================] - 0s 174us/step - loss: 0.3992 - accuracy: 0.7738 - val_loss: 0.3273 - val_accuracy: 0.8409
Epoch 734/1000
305/305 [==============================] - 0s 142us/step - loss: 0.3991 - accuracy: 0.7738 - val_loss: 0.3282 - val_accuracy: 0.8409
Epoch 735/1000
305/305 [==============================] - 0s 236us/step - loss: 0.3990 - accuracy: 0.7738 - val_loss: 0.3278 - val_accuracy: 0.8409
Epoch 736/1000
305/305 [==============================] - 0s 106us/step - loss: 0.3983 - accuracy: 0.7738 - val_loss: 0.3275 - 

305/305 [==============================] - 0s 167us/step - loss: 0.3600 - accuracy: 0.7738 - val_loss: 0.2971 - val_accuracy: 0.8409
Epoch 786/1000
305/305 [==============================] - 0s 135us/step - loss: 0.3556 - accuracy: 0.7738 - val_loss: 0.2912 - val_accuracy: 0.8409
Epoch 787/1000
305/305 [==============================] - 0s 116us/step - loss: 0.3544 - accuracy: 0.7738 - val_loss: 0.2813 - val_accuracy: 0.8409
Epoch 788/1000
305/305 [==============================] - 0s 125us/step - loss: 0.3611 - accuracy: 0.7738 - val_loss: 0.3398 - val_accuracy: 0.8409
Epoch 789/1000
305/305 [==============================] - 0s 112us/step - loss: 0.3704 - accuracy: 0.7738 - val_loss: 0.2978 - val_accuracy: 0.8409
Epoch 790/1000
305/305 [==============================] - 0s 100us/step - loss: 0.3561 - accuracy: 0.7738 - val_loss: 0.2977 - val_accuracy: 0.8409
Epoch 791/1000
305/305 [==============================] - 0s 99us/step - loss: 0.3555 - accuracy: 0.7738 - val_loss: 0.2937 - v

305/305 [==============================] - 0s 97us/step - loss: 0.4433 - accuracy: 0.7738 - val_loss: 0.3931 - val_accuracy: 0.8333
Epoch 841/1000
305/305 [==============================] - 0s 107us/step - loss: 0.4432 - accuracy: 0.7738 - val_loss: 0.3927 - val_accuracy: 0.8333
Epoch 842/1000
305/305 [==============================] - 0s 108us/step - loss: 0.4429 - accuracy: 0.7738 - val_loss: 0.3924 - val_accuracy: 0.8333
Epoch 843/1000
305/305 [==============================] - 0s 102us/step - loss: 0.4426 - accuracy: 0.7738 - val_loss: 0.3922 - val_accuracy: 0.8333
Epoch 844/1000
305/305 [==============================] - 0s 114us/step - loss: 0.4423 - accuracy: 0.7738 - val_loss: 0.3918 - val_accuracy: 0.8333
Epoch 845/1000
305/305 [==============================] - 0s 99us/step - loss: 0.4421 - accuracy: 0.7738 - val_loss: 0.3915 - val_accuracy: 0.8333
Epoch 846/1000
305/305 [==============================] - 0s 94us/step - loss: 0.4419 - accuracy: 0.7738 - val_loss: 0.3911 - val

305/305 [==============================] - 0s 102us/step - loss: 0.4303 - accuracy: 0.7738 - val_loss: 0.3775 - val_accuracy: 0.8258
Epoch 896/1000
305/305 [==============================] - 0s 278us/step - loss: 0.4313 - accuracy: 0.7738 - val_loss: 0.3775 - val_accuracy: 0.8258
Epoch 897/1000
305/305 [==============================] - 0s 169us/step - loss: 0.4295 - accuracy: 0.7738 - val_loss: 0.3766 - val_accuracy: 0.8258
Epoch 898/1000
305/305 [==============================] - 0s 124us/step - loss: 0.4301 - accuracy: 0.7738 - val_loss: 0.3770 - val_accuracy: 0.8258
Epoch 899/1000
305/305 [==============================] - 0s 101us/step - loss: 0.4287 - accuracy: 0.7738 - val_loss: 0.3764 - val_accuracy: 0.8258
Epoch 900/1000
305/305 [==============================] - 0s 86us/step - loss: 0.4286 - accuracy: 0.7738 - val_loss: 0.3764 - val_accuracy: 0.8258
Epoch 901/1000
305/305 [==============================] - 0s 126us/step - loss: 0.4284 - accuracy: 0.7738 - val_loss: 0.3761 - v

305/305 [==============================] - 0s 134us/step - loss: 0.4210 - accuracy: 0.7738 - val_loss: 0.3662 - val_accuracy: 0.8258
Epoch 951/1000
305/305 [==============================] - 0s 101us/step - loss: 0.4198 - accuracy: 0.7738 - val_loss: 0.3656 - val_accuracy: 0.8258
Epoch 952/1000
305/305 [==============================] - 0s 96us/step - loss: 0.4195 - accuracy: 0.7738 - val_loss: 0.3647 - val_accuracy: 0.8258
Epoch 953/1000
305/305 [==============================] - 0s 88us/step - loss: 0.4194 - accuracy: 0.7738 - val_loss: 0.3644 - val_accuracy: 0.8258
Epoch 954/1000
305/305 [==============================] - 0s 111us/step - loss: 0.4202 - accuracy: 0.7738 - val_loss: 0.3644 - val_accuracy: 0.8258
Epoch 955/1000
305/305 [==============================] - 0s 89us/step - loss: 0.4194 - accuracy: 0.7738 - val_loss: 0.3657 - val_accuracy: 0.8333
Epoch 956/1000
305/305 [==============================] - 0s 94us/step - loss: 0.4188 - accuracy: 0.7738 - val_loss: 0.3663 - val_

In [22]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

132/132 [==============================] - 0s 109us/step
combination test accuracy: 83.33%


In [23]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [24]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [25]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 336 samples, validate on 144 samples
Epoch 1/1000
336/336 [==============================] - 0s 728us/step - loss: 10.8218 - accuracy: 0.5417 - val_loss: 0.6350 - val_accuracy: 0.6111
Epoch 2/1000
336/336 [==============================] - 0s 145us/step - loss: 0.6332 - accuracy: 0.6726 - val_loss: 0.6223 - val_accuracy: 0.6528
Epoch 3/1000
336/336 [==============================] - 0s 143us/step - loss: 0.6158 - accuracy: 0.6845 - val_loss: 0.6175 - val_accuracy: 0.6528
Epoch 4/1000
336/336 [==============================] - 0s 141us/step - loss: 0.6128 - accuracy: 0.6905 - val_loss: 0.6142 - val_accuracy: 0.6528
Epoch 5/1000
336/336 [==============================] - 0s 121us/step - loss: 0.6067 - accuracy: 0.6845 - val_loss: 0.6069 - val_accuracy: 0.6458
Epoch 6/1000
336/336 [==============================] - 0s 152us/step - loss: 0.6013 - accuracy: 0.6726 - val_loss: 0.6007 - val_accuracy: 0.6528
Epoch 7/1000
336/336 [==============================] - 0s 119us/step - loss:

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.101719). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


336/336 [==============================] - 0s 402us/step - loss: 0.5366 - accuracy: 0.7173 - val_loss: 0.5350 - val_accuracy: 0.7778
Epoch 18/1000
336/336 [==============================] - 0s 177us/step - loss: 0.5216 - accuracy: 0.7560 - val_loss: 0.5325 - val_accuracy: 0.7778
Epoch 19/1000
336/336 [==============================] - 0s 104us/step - loss: 0.5268 - accuracy: 0.7619 - val_loss: 0.5246 - val_accuracy: 0.7708
Epoch 20/1000
336/336 [==============================] - 0s 102us/step - loss: 0.5301 - accuracy: 0.7411 - val_loss: 0.5228 - val_accuracy: 0.7708
Epoch 21/1000
336/336 [==============================] - 0s 156us/step - loss: 0.5086 - accuracy: 0.7679 - val_loss: 0.5185 - val_accuracy: 0.7708
Epoch 22/1000
336/336 [==============================] - 0s 161us/step - loss: 0.5084 - accuracy: 0.7649 - val_loss: 0.5166 - val_accuracy: 0.7778
Epoch 23/1000
336/336 [==============================] - 0s 136us/step - loss: 0.5020 - accuracy: 0.7708 - val_loss: 0.5150 - val_ac

Epoch 73/1000
336/336 [==============================] - 0s 125us/step - loss: 0.3806 - accuracy: 0.8363 - val_loss: 0.4001 - val_accuracy: 0.8403
Epoch 74/1000
336/336 [==============================] - 0s 99us/step - loss: 0.3709 - accuracy: 0.8333 - val_loss: 0.4085 - val_accuracy: 0.8403
Epoch 75/1000
336/336 [==============================] - 0s 116us/step - loss: 0.3718 - accuracy: 0.8452 - val_loss: 0.3987 - val_accuracy: 0.8403
Epoch 76/1000
336/336 [==============================] - 0s 125us/step - loss: 0.3729 - accuracy: 0.8393 - val_loss: 0.3913 - val_accuracy: 0.8403
Epoch 77/1000
336/336 [==============================] - 0s 99us/step - loss: 0.3646 - accuracy: 0.8452 - val_loss: 0.3920 - val_accuracy: 0.8403
Epoch 78/1000
336/336 [==============================] - 0s 96us/step - loss: 0.3650 - accuracy: 0.8363 - val_loss: 0.3925 - val_accuracy: 0.8403
Epoch 79/1000
336/336 [==============================] - 0s 93us/step - loss: 0.3610 - accuracy: 0.8452 - val_loss: 0.460

336/336 [==============================] - 0s 102us/step - loss: 0.3325 - accuracy: 0.8571 - val_loss: 0.4251 - val_accuracy: 0.8264
Epoch 129/1000
336/336 [==============================] - 0s 101us/step - loss: 0.3506 - accuracy: 0.8452 - val_loss: 0.3675 - val_accuracy: 0.8403
Epoch 130/1000
336/336 [==============================] - 0s 100us/step - loss: 0.3396 - accuracy: 0.8512 - val_loss: 0.3641 - val_accuracy: 0.8403
Epoch 131/1000
336/336 [==============================] - 0s 95us/step - loss: 0.3366 - accuracy: 0.8482 - val_loss: 0.3818 - val_accuracy: 0.8403
Epoch 132/1000
336/336 [==============================] - 0s 114us/step - loss: 0.3362 - accuracy: 0.8512 - val_loss: 0.3578 - val_accuracy: 0.8403
Epoch 133/1000
336/336 [==============================] - 0s 289us/step - loss: 0.3363 - accuracy: 0.8452 - val_loss: 0.3602 - val_accuracy: 0.8403
Epoch 134/1000
336/336 [==============================] - 0s 96us/step - loss: 0.3332 - accuracy: 0.8512 - val_loss: 0.3548 - va

...

Epoch 738/1000
336/336 [==============================] - 0s 105us/step - loss: 0.3137 - accuracy: 0.8542 - val_loss: 0.3355 - val_accuracy: 0.8403
Epoch 739/1000
336/336 [==============================] - 0s 86us/step - loss: 0.3153 - accuracy: 0.8512 - val_loss: 0.3332 - val_accuracy: 0.8403
Epoch 740/1000
336/336 [==============================] - 0s 88us/step - loss: 0.3196 - accuracy: 0.8452 - val_loss: 0.3332 - val_accuracy: 0.8403
Epoch 741/1000
336/336 [==============================] - 0s 85us/step - loss: 0.3128 - accuracy: 0.8542 - val_loss: 0.3342 - val_accuracy: 0.8403
Epoch 742/1000
336/336 [==============================] - 0s 127us/step - loss: 0.3164 - accuracy: 0.8512 - val_loss: 0.3352 - val_accuracy: 0.8403
Epoch 743/1000
336/336 [==============================] - 0s 128us/step - loss: 0.3147 - accuracy: 0.8512 - val_loss: 0.3341 - val_accuracy: 0.8403
Epoch 744/1000
336/336 [==============================] - 0s 98us/step - loss: 0.3127 - accuracy: 0.8512 - val_loss

336/336 [==============================] - 0s 95us/step - loss: 0.3143 - accuracy: 0.8512 - val_loss: 0.3339 - val_accuracy: 0.8403
Epoch 794/1000
336/336 [==============================] - 0s 132us/step - loss: 0.3141 - accuracy: 0.8512 - val_loss: 0.3336 - val_accuracy: 0.8403
Epoch 795/1000
336/336 [==============================] - 0s 122us/step - loss: 0.3140 - accuracy: 0.8512 - val_loss: 0.3331 - val_accuracy: 0.8403
Epoch 796/1000
336/336 [==============================] - 0s 84us/step - loss: 0.3139 - accuracy: 0.8512 - val_loss: 0.3327 - val_accuracy: 0.8403
Epoch 797/1000
336/336 [==============================] - 0s 120us/step - loss: 0.3126 - accuracy: 0.8512 - val_loss: 0.3325 - val_accuracy: 0.8403
Epoch 798/1000
336/336 [==============================] - 0s 86us/step - loss: 0.3120 - accuracy: 0.8571 - val_loss: 0.3327 - val_accuracy: 0.8403
Epoch 799/1000
336/336 [==============================] - 0s 88us/step - loss: 0.3192 - accuracy: 0.8393 - val_loss: 0.3387 - val_

Epoch 849/1000
336/336 [==============================] - 0s 122us/step - loss: 0.3112 - accuracy: 0.8512 - val_loss: 0.3299 - val_accuracy: 0.8403
Epoch 850/1000
336/336 [==============================] - 0s 106us/step - loss: 0.3120 - accuracy: 0.8452 - val_loss: 0.3289 - val_accuracy: 0.8403
Epoch 851/1000
336/336 [==============================] - 0s 140us/step - loss: 0.3145 - accuracy: 0.8512 - val_loss: 0.3316 - val_accuracy: 0.8403
Epoch 852/1000
336/336 [==============================] - 0s 135us/step - loss: 0.3146 - accuracy: 0.8452 - val_loss: 0.3290 - val_accuracy: 0.8403
Epoch 853/1000
336/336 [==============================] - 0s 90us/step - loss: 0.3078 - accuracy: 0.8542 - val_loss: 0.3516 - val_accuracy: 0.8056
Epoch 854/1000
336/336 [==============================] - 0s 91us/step - loss: 0.3264 - accuracy: 0.8452 - val_loss: 0.3332 - val_accuracy: 0.8403
Epoch 855/1000
336/336 [==============================] - 0s 91us/step - loss: 0.3144 - accuracy: 0.8512 - val_los

336/336 [==============================] - 0s 94us/step - loss: 0.3078 - accuracy: 0.8542 - val_loss: 0.3321 - val_accuracy: 0.8403
Epoch 905/1000
336/336 [==============================] - 0s 326us/step - loss: 0.3120 - accuracy: 0.8512 - val_loss: 0.3281 - val_accuracy: 0.8403
Epoch 906/1000
336/336 [==============================] - 0s 87us/step - loss: 0.3119 - accuracy: 0.8512 - val_loss: 0.3289 - val_accuracy: 0.8403
Epoch 907/1000
336/336 [==============================] - 0s 100us/step - loss: 0.3100 - accuracy: 0.8571 - val_loss: 0.3264 - val_accuracy: 0.8472
Epoch 908/1000
336/336 [==============================] - 0s 90us/step - loss: 0.3102 - accuracy: 0.8542 - val_loss: 0.3278 - val_accuracy: 0.8403
Epoch 909/1000
336/336 [==============================] - 0s 89us/step - loss: 0.3073 - accuracy: 0.8601 - val_loss: 0.3262 - val_accuracy: 0.8472
Epoch 910/1000
336/336 [==============================] - 0s 89us/step - loss: 0.3083 - accuracy: 0.8571 - val_loss: 0.3293 - val_a

Epoch 960/1000
336/336 [==============================] - 0s 92us/step - loss: 0.3108 - accuracy: 0.8512 - val_loss: 0.3307 - val_accuracy: 0.8403
Epoch 961/1000
336/336 [==============================] - 0s 96us/step - loss: 0.3196 - accuracy: 0.8363 - val_loss: 0.3339 - val_accuracy: 0.8403
Epoch 962/1000
336/336 [==============================] - 0s 95us/step - loss: 0.3136 - accuracy: 0.8512 - val_loss: 0.3326 - val_accuracy: 0.8403
Epoch 963/1000
336/336 [==============================] - 0s 90us/step - loss: 0.3133 - accuracy: 0.8512 - val_loss: 0.3321 - val_accuracy: 0.8403
Epoch 964/1000
336/336 [==============================] - 0s 88us/step - loss: 0.3129 - accuracy: 0.8512 - val_loss: 0.3309 - val_accuracy: 0.8403
Epoch 965/1000
336/336 [==============================] - 0s 92us/step - loss: 0.3111 - accuracy: 0.8512 - val_loss: 0.3303 - val_accuracy: 0.8472
Epoch 966/1000
336/336 [==============================] - 0s 98us/step - loss: 0.3087 - accuracy: 0.8542 - val_loss: 0

In [26]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

144/144 [==============================] - 0s 77us/step
over-sampling test accuracy: 81.94%


In [27]:
############ Feature selection using lasso ##########

In [28]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [29]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
selection.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [30]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [31]:
X_train_features_comb = np.vstack((names, X_train_comb))
X_train_features_comb = pd.DataFrame(X_train_features_comb)

In [32]:
sel_features = X_train_features_comb.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_comb.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 866
selected features: 41


In [33]:
cols = sel_features.values
cols.reshape((1, -1))

array([[  4,  12,  29,  55,  91, 145, 157, 175, 274, 276, 285, 353, 362,
        371, 415, 416, 490, 541, 545, 549, 552, 571, 600, 622, 644, 645,
        655, 672, 673, 685, 689, 694, 695, 697, 709, 711, 726, 729, 741,
        864, 865]])

In [34]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA',
       'TTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAAT',
       'TTTTATAGTACTTTTTTAAACTTTGCTATATCTCTTGTGTTATGA',
       'TTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAATCA',
       'TTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAATCATATTATTT',
       'TTATCGACAGAT', 'TTACTGACAAG', 'TTAACATAATAAT',
       'TCATATTCAATCTCCCTTAAATATCACTATGACATTTACGCTTTACATCATCATATGCGCACAAATGCTCATTACTTTTTTATAGATACAAATTTAGTAT',
       'TCATAACACAAGAGATATAGCAAAGTTTAAAAAAGTACTATAAAATAGCAATTGAATGTCCAGTAACAAATTTGGAGGAAGCGTATATGTATCAAACAAT',
       'TCAATCTCCCTTAAATATCACTATGACATTTACGCTTTACATCATCATATGCGCACAAATGCTCATTACTTTTTTATAGATACAAATTTAGTATTATTAT',
       'TAATTGTTTGATACATATACGCTTCCTCCAAATTTGTTACTGGACATTCAATTGCTATTTTATAGTACTTTTTTAAACTTTGCTATATCTCTTGTGTTAT',
 

In [36]:
removed_features = X_train_features_comb.columns[(selection.estimator_.coef_ == 0).ravel().tolist()]
removed_features

Int64Index([  0,   1,   2,   3,   5,   6,   7,   8,   9,  10,
            ...
            854, 855, 856, 857, 858, 859, 860, 861, 862, 863],
           dtype='int64', length=825)

In [37]:
len(removed_features)

825

In [38]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']
X_sel = df_sel.loc[:, df_sel.columns != 'pheno'].values
y_sel = df_sel['pheno'].values
print(X_sel.shape, y_sel.shape, df_sel.shape)

(255, 41) (255,) (255, 42)


In [39]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_sel_comb, y_sel_comb = smote_enn.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_comb).items()))

[(0, 209), (1, 213)]


In [40]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 240), (1, 240)]


In [41]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_sel_train_comb, X_sel_test_comb, y_sel_train_comb, y_sel_test_comb = train_test_split(X_sel_comb, y_sel_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_comb)

In [42]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_over)

In [43]:
#### neural network on combination data
model2_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [44]:
model2_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [45]:
model2_comb.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 295 samples, validate on 127 samples
Epoch 1/1000
295/295 [==============================] - 0s 1ms/step - loss: 9.5319 - accuracy: 0.4814 - val_loss: 0.6862 - val_accuracy: 0.5039
Epoch 2/1000
295/295 [==============================] - 0s 123us/step - loss: 0.6796 - accuracy: 0.5559 - val_loss: 0.6656 - val_accuracy: 0.5118
Epoch 3/1000
295/295 [==============================] - 0s 99us/step - loss: 0.6418 - accuracy: 0.5831 - val_loss: 0.6218 - val_accuracy: 0.7953
Epoch 4/1000
295/295 [==============================] - 0s 138us/step - loss: 0.6325 - accuracy: 0.7288 - val_loss: 0.6411 - val_accuracy: 0.7165
Epoch 5/1000
295/295 [==============================] - 0s 241us/step - loss: 0.6381 - accuracy: 0.6847 - val_loss: 0.5834 - val_accuracy: 0.8425
Epoch 6/1000
295/295 [==============================] - 0s 134us/step - loss: 0.6093 - accuracy: 0.7220 - val_loss: 0.5629 - val_accuracy: 0.8110
Epoch 7/1000
295/295 [==============================] - 0s 127us/step - loss: 0.5

295/295 [==============================] - 0s 117us/step - loss: 0.4402 - accuracy: 0.8102 - val_loss: 0.3890 - val_accuracy: 0.8583
Epoch 57/1000
295/295 [==============================] - 0s 143us/step - loss: 0.4247 - accuracy: 0.8237 - val_loss: 0.3859 - val_accuracy: 0.8583
Epoch 58/1000
295/295 [==============================] - 0s 138us/step - loss: 0.4155 - accuracy: 0.8407 - val_loss: 0.5087 - val_accuracy: 0.7402
Epoch 59/1000
295/295 [==============================] - 0s 107us/step - loss: 0.4186 - accuracy: 0.8271 - val_loss: 0.3775 - val_accuracy: 0.8583
Epoch 60/1000
295/295 [==============================] - 0s 88us/step - loss: 0.4049 - accuracy: 0.8407 - val_loss: 0.3766 - val_accuracy: 0.8583
Epoch 61/1000
295/295 [==============================] - 0s 84us/step - loss: 0.4040 - accuracy: 0.8407 - val_loss: 0.3784 - val_accuracy: 0.8583
Epoch 62/1000
295/295 [==============================] - 0s 133us/step - loss: 0.4032 - accuracy: 0.8407 - val_loss: 0.3685 - val_accu

Epoch 112/1000
295/295 [==============================] - 0s 124us/step - loss: 0.4139 - accuracy: 0.8271 - val_loss: 0.3464 - val_accuracy: 0.8583
Epoch 113/1000
295/295 [==============================] - 0s 148us/step - loss: 0.4054 - accuracy: 0.8271 - val_loss: 0.3413 - val_accuracy: 0.8346
Epoch 114/1000
295/295 [==============================] - 0s 113us/step - loss: 0.4091 - accuracy: 0.8271 - val_loss: 0.3395 - val_accuracy: 0.8583
Epoch 115/1000
295/295 [==============================] - 0s 84us/step - loss: 0.3975 - accuracy: 0.8373 - val_loss: 0.3463 - val_accuracy: 0.8583
Epoch 116/1000
295/295 [==============================] - 0s 83us/step - loss: 0.3995 - accuracy: 0.8305 - val_loss: 0.3455 - val_accuracy: 0.8583
Epoch 117/1000
295/295 [==============================] - 0s 113us/step - loss: 0.3884 - accuracy: 0.8407 - val_loss: 0.3833 - val_accuracy: 0.8346
Epoch 118/1000
295/295 [==============================] - 0s 80us/step - loss: 0.4303 - accuracy: 0.8136 - val_los

295/295 [==============================] - 0s 104us/step - loss: 0.3429 - accuracy: 0.8407 - val_loss: 0.3130 - val_accuracy: 0.8583
Epoch 168/1000
295/295 [==============================] - 0s 88us/step - loss: 0.3405 - accuracy: 0.8407 - val_loss: 0.3274 - val_accuracy: 0.8583
Epoch 169/1000
295/295 [==============================] - 0s 87us/step - loss: 0.3417 - accuracy: 0.8407 - val_loss: 0.3080 - val_accuracy: 0.8583
Epoch 170/1000
295/295 [==============================] - 0s 88us/step - loss: 0.3633 - accuracy: 0.8271 - val_loss: 0.3081 - val_accuracy: 0.8583
Epoch 171/1000
295/295 [==============================] - 0s 87us/step - loss: 0.3402 - accuracy: 0.8407 - val_loss: 0.3206 - val_accuracy: 0.8583
Epoch 172/1000
295/295 [==============================] - 0s 85us/step - loss: 0.3406 - accuracy: 0.8407 - val_loss: 0.3058 - val_accuracy: 0.8583
Epoch 173/1000
295/295 [==============================] - 0s 85us/step - loss: 0.3397 - accuracy: 0.8407 - val_loss: 0.3306 - val_ac

295/295 [==============================] - 0s 99us/step - loss: 0.2928 - accuracy: 0.8508 - val_loss: 0.2627 - val_accuracy: 0.8898
Epoch 223/1000
295/295 [==============================] - 0s 185us/step - loss: 0.3000 - accuracy: 0.8542 - val_loss: 0.2659 - val_accuracy: 0.8898
Epoch 224/1000
295/295 [==============================] - 0s 144us/step - loss: 0.2974 - accuracy: 0.8542 - val_loss: 0.2639 - val_accuracy: 0.8898
Epoch 225/1000
295/295 [==============================] - 0s 96us/step - loss: 0.2947 - accuracy: 0.8542 - val_loss: 0.2652 - val_accuracy: 0.8898
Epoch 226/1000
295/295 [==============================] - 0s 106us/step - loss: 0.2938 - accuracy: 0.8542 - val_loss: 0.2568 - val_accuracy: 0.8898
Epoch 227/1000
295/295 [==============================] - 0s 118us/step - loss: 0.2930 - accuracy: 0.8542 - val_loss: 0.2844 - val_accuracy: 0.8661
Epoch 228/1000
295/295 [==============================] - 0s 87us/step - loss: 1.3317 - accuracy: 0.7288 - val_loss: 0.3934 - val

...

295/295 [==============================] - 0s 88us/step - loss: 0.4087 - accuracy: 0.8339 - val_loss: 0.3219 - val_accuracy: 0.8740
Epoch 776/1000
295/295 [==============================] - 0s 214us/step - loss: 0.3401 - accuracy: 0.8644 - val_loss: 0.3324 - val_accuracy: 0.8740
Epoch 777/1000
295/295 [==============================] - 0s 138us/step - loss: 0.3328 - accuracy: 0.8678 - val_loss: 0.3053 - val_accuracy: 0.8819
Epoch 778/1000
295/295 [==============================] - 0s 83us/step - loss: 0.4377 - accuracy: 0.8068 - val_loss: 0.3325 - val_accuracy: 0.8819
Epoch 779/1000
295/295 [==============================] - 0s 123us/step - loss: 0.3745 - accuracy: 0.8542 - val_loss: 0.3330 - val_accuracy: 0.8819
Epoch 780/1000
295/295 [==============================] - 0s 92us/step - loss: 0.3700 - accuracy: 0.8576 - val_loss: 0.3291 - val_accuracy: 0.8819
Epoch 781/1000
295/295 [==============================] - 0s 82us/step - loss: 0.3801 - accuracy: 0.8339 - val_loss: 0.3450 - val_

Epoch 831/1000
295/295 [==============================] - 0s 94us/step - loss: 0.3595 - accuracy: 0.8542 - val_loss: 0.3156 - val_accuracy: 0.8819
Epoch 832/1000
295/295 [==============================] - 0s 87us/step - loss: 0.3339 - accuracy: 0.8678 - val_loss: 0.2944 - val_accuracy: 0.8819
Epoch 833/1000
295/295 [==============================] - 0s 78us/step - loss: 0.3101 - accuracy: 0.8678 - val_loss: 0.2908 - val_accuracy: 0.8819
Epoch 834/1000
295/295 [==============================] - 0s 126us/step - loss: 0.3080 - accuracy: 0.8712 - val_loss: 0.3004 - val_accuracy: 0.8819
Epoch 835/1000
295/295 [==============================] - 0s 89us/step - loss: 0.3099 - accuracy: 0.8712 - val_loss: 0.2906 - val_accuracy: 0.8819
Epoch 836/1000
295/295 [==============================] - 0s 174us/step - loss: 0.3088 - accuracy: 0.8678 - val_loss: 0.4610 - val_accuracy: 0.7874
Epoch 837/1000
295/295 [==============================] - 0s 85us/step - loss: 0.3792 - accuracy: 0.8339 - val_loss:

295/295 [==============================] - 0s 91us/step - loss: 0.2954 - accuracy: 0.8712 - val_loss: 0.2820 - val_accuracy: 0.8819
Epoch 887/1000
295/295 [==============================] - 0s 127us/step - loss: 0.2954 - accuracy: 0.8712 - val_loss: 0.2815 - val_accuracy: 0.8819
Epoch 888/1000
295/295 [==============================] - 0s 81us/step - loss: 0.2970 - accuracy: 0.8712 - val_loss: 0.2888 - val_accuracy: 0.8819
Epoch 889/1000
295/295 [==============================] - 0s 127us/step - loss: 0.2974 - accuracy: 0.8678 - val_loss: 0.2846 - val_accuracy: 0.8819
Epoch 890/1000
295/295 [==============================] - 0s 83us/step - loss: 0.2935 - accuracy: 0.8712 - val_loss: 0.2807 - val_accuracy: 0.8819
Epoch 891/1000
295/295 [==============================] - 0s 85us/step - loss: 0.3014 - accuracy: 0.8644 - val_loss: 0.2864 - val_accuracy: 0.8819
Epoch 892/1000
295/295 [==============================] - 0s 94us/step - loss: 0.3101 - accuracy: 0.8644 - val_loss: 0.2813 - val_a

Epoch 942/1000
295/295 [==============================] - 0s 87us/step - loss: 0.2971 - accuracy: 0.8644 - val_loss: 0.2784 - val_accuracy: 0.8819
Epoch 943/1000
295/295 [==============================] - 0s 83us/step - loss: 0.2891 - accuracy: 0.8678 - val_loss: 0.3007 - val_accuracy: 0.8740
Epoch 944/1000
295/295 [==============================] - 0s 93us/step - loss: 0.2908 - accuracy: 0.8678 - val_loss: 0.2878 - val_accuracy: 0.8819
Epoch 945/1000
295/295 [==============================] - 0s 89us/step - loss: 0.2901 - accuracy: 0.8678 - val_loss: 0.2793 - val_accuracy: 0.8819
Epoch 946/1000
295/295 [==============================] - 0s 180us/step - loss: 0.2879 - accuracy: 0.8712 - val_loss: 0.2857 - val_accuracy: 0.8819
Epoch 947/1000
295/295 [==============================] - 0s 125us/step - loss: 0.2904 - accuracy: 0.8712 - val_loss: 0.2781 - val_accuracy: 0.8819
Epoch 948/1000
295/295 [==============================] - 0s 105us/step - loss: 0.2886 - accuracy: 0.8712 - val_loss

295/295 [==============================] - 0s 289us/step - loss: 0.2850 - accuracy: 0.8712 - val_loss: 0.2769 - val_accuracy: 0.8819
Epoch 998/1000
295/295 [==============================] - 0s 92us/step - loss: 0.2846 - accuracy: 0.8712 - val_loss: 0.2788 - val_accuracy: 0.8819
Epoch 999/1000
295/295 [==============================] - 0s 121us/step - loss: 0.2845 - accuracy: 0.8712 - val_loss: 0.2848 - val_accuracy: 0.8819
Epoch 1000/1000
295/295 [==============================] - 0s 106us/step - loss: 0.2846 - accuracy: 0.8712 - val_loss: 0.2757 - val_accuracy: 0.8819


In [46]:
acc_test_comb2 = model2_comb.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))


127/127 [==============================] - 0s 288us/step
combination test accuracy: 88.19%


In [47]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [48]:
model2_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [49]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 336 samples, validate on 144 samples
Epoch 1/1000
336/336 [==============================] - 0s 356us/step - loss: 40.2209 - accuracy: 0.5833 - val_loss: 0.6894 - val_accuracy: 0.6319
Epoch 2/1000
336/336 [==============================] - 0s 105us/step - loss: 0.6750 - accuracy: 0.6399 - val_loss: 0.6447 - val_accuracy: 0.6250
Epoch 3/1000
336/336 [==============================] - 0s 268us/step - loss: 0.6583 - accuracy: 0.6607 - val_loss: 0.6403 - val_accuracy: 0.6736
Epoch 4/1000
336/336 [==============================] - 0s 244us/step - loss: 0.6530 - accuracy: 0.6577 - val_loss: 0.6375 - val_accuracy: 0.6250
Epoch 5/1000
336/336 [==============================] - 0s 122us/step - loss: 0.6498 - accuracy: 0.6548 - val_loss: 0.6356 - val_accuracy: 0.6250
Epoch 6/1000
336/336 [==============================] - 0s 99us/step - loss: 0.6435 - accuracy: 0.6488 - val_loss: 0.6346 - val_accuracy: 0.6250
Epoch 7/1000
336/336 [==============================] - 0s 128us/step - loss: 

336/336 [==============================] - 0s 130us/step - loss: 0.5471 - accuracy: 0.7262 - val_loss: 0.5353 - val_accuracy: 0.7083
Epoch 57/1000
336/336 [==============================] - 0s 128us/step - loss: 0.5242 - accuracy: 0.7381 - val_loss: 0.5441 - val_accuracy: 0.7083
Epoch 58/1000
336/336 [==============================] - 0s 95us/step - loss: 0.5252 - accuracy: 0.7381 - val_loss: 0.5360 - val_accuracy: 0.7014
Epoch 59/1000
336/336 [==============================] - 0s 90us/step - loss: 0.5268 - accuracy: 0.7381 - val_loss: 0.5632 - val_accuracy: 0.6319
Epoch 60/1000
336/336 [==============================] - 0s 82us/step - loss: 0.5226 - accuracy: 0.7321 - val_loss: 0.5324 - val_accuracy: 0.7083
Epoch 61/1000
336/336 [==============================] - 0s 122us/step - loss: 0.5217 - accuracy: 0.7321 - val_loss: 0.5292 - val_accuracy: 0.7083
Epoch 62/1000
336/336 [==============================] - 0s 75us/step - loss: 0.5191 - accuracy: 0.7351 - val_loss: 0.5299 - val_accura

336/336 [==============================] - 0s 81us/step - loss: 0.4815 - accuracy: 0.7798 - val_loss: 0.5399 - val_accuracy: 0.6458
Epoch 113/1000
336/336 [==============================] - 0s 85us/step - loss: 0.4739 - accuracy: 0.7649 - val_loss: 0.4946 - val_accuracy: 0.7292
Epoch 114/1000
336/336 [==============================] - 0s 85us/step - loss: 0.4600 - accuracy: 0.7768 - val_loss: 0.5126 - val_accuracy: 0.7292
Epoch 115/1000
336/336 [==============================] - 0s 82us/step - loss: 0.4669 - accuracy: 0.7768 - val_loss: 0.4994 - val_accuracy: 0.7292
Epoch 116/1000
336/336 [==============================] - 0s 78us/step - loss: 0.4652 - accuracy: 0.7768 - val_loss: 0.5106 - val_accuracy: 0.7292
Epoch 117/1000
336/336 [==============================] - 0s 83us/step - loss: 0.4619 - accuracy: 0.7827 - val_loss: 0.4917 - val_accuracy: 0.7778
Epoch 118/1000
336/336 [==============================] - 0s 114us/step - loss: 0.4608 - accuracy: 0.7768 - val_loss: 0.5017 - val_ac

336/336 [==============================] - 0s 83us/step - loss: 0.4831 - accuracy: 0.7976 - val_loss: 0.5122 - val_accuracy: 0.7778
Epoch 168/1000
336/336 [==============================] - 0s 140us/step - loss: 0.4769 - accuracy: 0.8065 - val_loss: 0.5106 - val_accuracy: 0.7500
Epoch 169/1000
336/336 [==============================] - 0s 145us/step - loss: 0.4818 - accuracy: 0.7857 - val_loss: 0.5167 - val_accuracy: 0.7500
Epoch 170/1000
336/336 [==============================] - 0s 107us/step - loss: 0.4745 - accuracy: 0.8065 - val_loss: 0.5028 - val_accuracy: 0.7569
Epoch 171/1000
336/336 [==============================] - 0s 133us/step - loss: 0.4826 - accuracy: 0.7946 - val_loss: 0.6029 - val_accuracy: 0.7222
Epoch 172/1000
336/336 [==============================] - 0s 70us/step - loss: 0.4865 - accuracy: 0.7917 - val_loss: 0.5055 - val_accuracy: 0.7847
Epoch 173/1000
336/336 [==============================] - 0s 77us/step - loss: 0.4917 - accuracy: 0.7976 - val_loss: 0.5581 - val

Epoch 223/1000
336/336 [==============================] - 0s 274us/step - loss: 0.4165 - accuracy: 0.8125 - val_loss: 0.4620 - val_accuracy: 0.7847
Epoch 224/1000
336/336 [==============================] - 0s 227us/step - loss: 0.4218 - accuracy: 0.8036 - val_loss: 0.5219 - val_accuracy: 0.7083
Epoch 225/1000
336/336 [==============================] - 0s 205us/step - loss: 0.4303 - accuracy: 0.7917 - val_loss: 0.4624 - val_accuracy: 0.7847
Epoch 226/1000
336/336 [==============================] - 0s 269us/step - loss: 0.4211 - accuracy: 0.7976 - val_loss: 0.4504 - val_accuracy: 0.7847
Epoch 227/1000
336/336 [==============================] - 0s 90us/step - loss: 0.4135 - accuracy: 0.8095 - val_loss: 0.5106 - val_accuracy: 0.7083
Epoch 228/1000
336/336 [==============================] - 0s 162us/step - loss: 0.4157 - accuracy: 0.8065 - val_loss: 0.6223 - val_accuracy: 0.7222
Epoch 229/1000
336/336 [==============================] - 0s 195us/step - loss: 0.4191 - accuracy: 0.8095 - val_l

...

Epoch 777/1000
336/336 [==============================] - 0s 86us/step - loss: 0.3367 - accuracy: 0.8363 - val_loss: 0.3666 - val_accuracy: 0.8125
Epoch 778/1000
336/336 [==============================] - 0s 87us/step - loss: 0.3406 - accuracy: 0.8304 - val_loss: 0.3644 - val_accuracy: 0.8125
Epoch 779/1000
336/336 [==============================] - 0s 82us/step - loss: 0.3392 - accuracy: 0.8393 - val_loss: 0.3643 - val_accuracy: 0.8125
Epoch 780/1000
336/336 [==============================] - 0s 89us/step - loss: 0.3431 - accuracy: 0.8363 - val_loss: 0.3688 - val_accuracy: 0.8125
Epoch 781/1000
336/336 [==============================] - 0s 89us/step - loss: 0.3387 - accuracy: 0.8363 - val_loss: 0.3813 - val_accuracy: 0.8125
Epoch 782/1000
336/336 [==============================] - 0s 80us/step - loss: 0.3431 - accuracy: 0.8304 - val_loss: 0.4176 - val_accuracy: 0.7708
Epoch 783/1000
336/336 [==============================] - 0s 116us/step - loss: 0.3572 - accuracy: 0.8244 - val_loss: 

336/336 [==============================] - 0s 81us/step - loss: 0.3314 - accuracy: 0.8393 - val_loss: 0.3614 - val_accuracy: 0.8125
Epoch 833/1000
336/336 [==============================] - 0s 70us/step - loss: 0.3294 - accuracy: 0.8363 - val_loss: 0.3602 - val_accuracy: 0.8125
Epoch 834/1000
336/336 [==============================] - 0s 70us/step - loss: 0.3375 - accuracy: 0.8393 - val_loss: 0.3687 - val_accuracy: 0.8125
Epoch 835/1000
336/336 [==============================] - 0s 80us/step - loss: 0.3332 - accuracy: 0.8393 - val_loss: 0.3598 - val_accuracy: 0.8125
Epoch 836/1000
336/336 [==============================] - 0s 79us/step - loss: 0.3259 - accuracy: 0.8393 - val_loss: 0.3628 - val_accuracy: 0.8125
Epoch 837/1000
336/336 [==============================] - 0s 88us/step - loss: 0.3287 - accuracy: 0.8393 - val_loss: 0.3591 - val_accuracy: 0.8125
Epoch 838/1000
336/336 [==============================] - 0s 131us/step - loss: 0.3236 - accuracy: 0.8363 - val_loss: 0.3587 - val_ac

Epoch 888/1000
336/336 [==============================] - 0s 306us/step - loss: 0.3258 - accuracy: 0.8423 - val_loss: 0.3540 - val_accuracy: 0.8194
Epoch 889/1000
336/336 [==============================] - 0s 162us/step - loss: 0.3332 - accuracy: 0.8393 - val_loss: 0.3548 - val_accuracy: 0.8194
Epoch 890/1000
336/336 [==============================] - 0s 249us/step - loss: 0.3315 - accuracy: 0.8423 - val_loss: 0.3562 - val_accuracy: 0.8194
Epoch 891/1000
336/336 [==============================] - 0s 109us/step - loss: 0.3270 - accuracy: 0.8393 - val_loss: 0.3660 - val_accuracy: 0.8194
Epoch 892/1000
336/336 [==============================] - 0s 134us/step - loss: 0.3275 - accuracy: 0.8423 - val_loss: 0.4465 - val_accuracy: 0.7361
Epoch 893/1000
336/336 [==============================] - 0s 177us/step - loss: 0.3361 - accuracy: 0.8333 - val_loss: 0.3559 - val_accuracy: 0.8194
Epoch 894/1000
336/336 [==============================] - 0s 92us/step - loss: 0.3273 - accuracy: 0.8393 - val_l

336/336 [==============================] - 0s 123us/step - loss: 0.3608 - accuracy: 0.8393 - val_loss: 0.3886 - val_accuracy: 0.8333
Epoch 944/1000
336/336 [==============================] - 0s 153us/step - loss: 0.3539 - accuracy: 0.8512 - val_loss: 0.3888 - val_accuracy: 0.8333
Epoch 945/1000
336/336 [==============================] - 0s 111us/step - loss: 0.3586 - accuracy: 0.8482 - val_loss: 0.3815 - val_accuracy: 0.8333
Epoch 946/1000
336/336 [==============================] - 0s 123us/step - loss: 0.3518 - accuracy: 0.8512 - val_loss: 0.3763 - val_accuracy: 0.8333
Epoch 947/1000
336/336 [==============================] - 0s 116us/step - loss: 0.3588 - accuracy: 0.8393 - val_loss: 0.4027 - val_accuracy: 0.8125
Epoch 948/1000
336/336 [==============================] - 0s 104us/step - loss: 0.3588 - accuracy: 0.8452 - val_loss: 0.3753 - val_accuracy: 0.8333
Epoch 949/1000
336/336 [==============================] - 0s 99us/step - loss: 0.3547 - accuracy: 0.8482 - val_loss: 0.3826 - v

336/336 [==============================] - 0s 122us/step - loss: 0.3418 - accuracy: 0.8452 - val_loss: 0.3638 - val_accuracy: 0.8403
Epoch 999/1000
336/336 [==============================] - 0s 113us/step - loss: 0.3426 - accuracy: 0.8571 - val_loss: 0.4430 - val_accuracy: 0.6875
Epoch 1000/1000
336/336 [==============================] - 0s 166us/step - loss: 0.3473 - accuracy: 0.8363 - val_loss: 0.3626 - val_accuracy: 0.8403


In [50]:
acc_test_over2 = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

144/144 [==============================] - 0s 293us/step
over-sampling test accuracy: 84.03%


In [51]:
############ Logistic Regression ############

In [52]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [53]:
###### logistics on combination data
log_comb = LogisticRegression()
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [54]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 100.00%


In [55]:
##### logistics on over-sampling data
log_over = LogisticRegression()
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [56]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 97.92%


In [57]:
############## Random Forest ##############

In [58]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [59]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 99.24%


In [60]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [61]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 100.00%


In [62]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [63]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.         0.96721311 1.         1.         1.        ]
0.9934426229508198


In [64]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.98529412 0.97058824 1.         1.         0.96969697]
0.9851158645276292
